In [1]:
import concurrent.futures
import time
import sys
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler,EmbeddingComposite,DWaveSampler 
from dwave.samplers import SimulatedAnnealingSampler
import pandas as pd
import numpy as np
import seaborn as sns
from distance_matrix_creator import *
import matplotlib.pyplot as plt
import dwave.inspector

In [2]:
# np.set_printoptions(suppress=True,threshold= sys.maxsize, linewidth=1000,formatter={'float_kind':'{}'.format, 'all': lambda x: " {:.0f}. ".format(x)})
np.set_printoptions(linewidth=1000, suppress=True,threshold= sys.maxsize)
flow = pd.read_csv("csv_files/flowmatrixbqm.csv", header = None)
# flow = (flow.to_numpy()/3)**5
flow = (flow.to_numpy())**3
flow = np.round(flow)
flow = np.triu(flow)
flow = np.array(flow)
print(flow,"\n\n")


distance = squaredistmatrix(9)
# distance = np.round(distance,decimals=3)
# distance = np.triu(np.array(distance))
distance = (np.array(distance))#**3)/5
# distance = np.triu(distance)
distance = np.round(distance,decimals=2)
print(distance)
print(np.shape(distance))

save = pd.DataFrame(np.round(distance, decimals=3))
save.to_csv("distance_matrix.csv")

Nfacil = 13
Npos = 81
matrL =9 
positions = []
facilities = [i for i in range(Nfacil)]
positions = [i for i in range(Npos)]

print("Facilities = ", facilities)
print("Positions = ",positions)

facility_size = [10,4,7,4,2,1,14,2,14,3,1,3,2]

[[  0 216   8   8   8   8   8   8   8   8   8   8   8]
 [  0   0 216 216 216  27  27  64   8  64  64  27  64]
 [  0   0   0  64 125  27   8   8   8   8   8 -64   8]
 [  0   0   0   0  27 125   8  64   8   8   8 -64   8]
 [  0   0   0   0   0  64 125  27   8   8   8 -64   8]
 [  0   0   0   0   0   0  27 125   8  27   8 -64   8]
 [  0   0   0   0   0   0   0  64 125  27   8   8   8]
 [  0   0   0   0   0   0   0   0 125  27  27   8   8]
 [  0   0   0   0   0   0   0   0   0 125 125  27  27]
 [  0   0   0   0   0   0   0   0   0   0  64  27  64]
 [  0   0   0   0   0   0   0   0   0   0   0  27  64]
 [  0   0   0   0   0   0   0   0   0   0   0   0  27]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]] 


[[ 0.    1.    2.    3.    4.    5.    6.    7.    8.    1.    1.41  2.24  3.16  4.12  5.1   6.08  7.07  8.06  2.    2.24  2.83  3.61  4.47  5.39  6.32  7.28  8.25  3.    3.16  3.61  4.24  5.    5.83  6.71  7.62  8.54  4.    4.12  4.47  5.    5.66  6.4   7.21  8.06  8.94  5.    5.

#Building a variable for each Machine

In [3]:
x = []
for f in facilities:
    x.append([f'F{f}P{p}' for p in positions])
# print(np.array(x))

#Initialise BQM

In [4]:
bqm = BinaryQuadraticModel('BINARY')

#Objective function

In [5]:
for f in range(len(facilities)):
    for f1 in range(len(facilities)):     
        for p in range(len(positions)):
            for p1 in range(len(positions)):
                if f == f1 and p == p1:
                    pass
                else:
                    bqm.add_quadratic(x[f][p],x[f1][p1],distance[p][p1]*flow[f][f1])
                    # print(f"{x[f][p]},{x[f1][p1]},{distance[p][p1]*flow[f][f1]}")

#constraint 1: only 1 machine is placed per position

In [6]:
for p in positions:
    c1 = [(x[f][p],1) for f in facilities]
    bqm.add_linear_inequality_constraint(
        c1,
        ub = 1,
        lb = 0,
        lagrange_multiplier=15000,
        label = 'c1_posi_' + str(p),
    )

#Constraint 2: Each facility is given correct size

In [7]:
for f in facilities:
    c2 = [(x[f][p],1) for p in positions]
    bqm.add_linear_equality_constraint(
        c2,
        constant=-1*facility_size[f],
        lagrange_multiplier=15000,
        # label = "c2_facil_" + str(f)
    )

Removing 0 bias variables and couplers from BQM

In [8]:
new_bqm = BinaryQuadraticModel(bqm.linear, {interaction: bias for interaction, bias in bqm.quadratic.items() if bias}, bqm.offset, bqm.vartype)
file = open("txt_files/bqm.txt", "w")
file.write(str(bqm))
file.close()

file2 = open("txt_files/new_bqm.txt", "w")
file2.write(str(new_bqm))
file2.close()

#running the solver

In [9]:
sampler = LeapHybridSampler()
# sampler = EmbeddingComposite(DWaveSampler())
# sampler = SimulatedAnnealingSampler()
numreads = 1000000 #number of samples for simulated annealer
timelimit = 90 #time limit for hybrid sampler

t1 = time.time()
#FOR HYBRID SAMPLER
sampleset = sampler.sample(new_bqm, time_limit = timelimit)
t2 = time.time()
print(f'solver finished in {t2-t1} seconds') 

solver finished in 31.799017429351807 seconds


Printing Output Solutions

In [11]:
t3 = time.time()
# print(samplesets[0])
# ### Extracting best run from all samplesets
# energies = [sampleset.first.energy for sampleset in samplesets]
# print(energies)
# sampleset = samplesets[energies.index(np.min(energies))]
# print(sampleset.first.sample)

# dwave.inspector.show(sampleset)

### Post Processing the input matrix to convert to a more readable cartesian layout
print("energy = ",sampleset.first.energy)
printout = []
for f in facilities:
    printouttemp = []
    for p in positions:
        label = f'F{f}P{p}'
        value = sampleset.first.sample[label]
        printouttemp.append(value) 
    printout.append(printouttemp)
print(printout)
layout = np.zeros((matrL,matrL))
ctr = 1
for i in printout:
    for j in range(len(i)):
        if i[j] == 1:
            q = int(j/len(layout))
            r = j%len(layout)
            layout[q][r] = ctr
    ctr+=1

### Plotting the Layout using Heatmap
fig, ax = plt.subplots(figsize=(9, 10))
sns.heatmap(layout,annot = layout, vmin = 0, vmax = len(facilities)).set(title = "Final Layout")
plt.savefig(f'Images/1HS_layout_timelimit_{timelimit}_IF_200.png')

### Check to ensure all facilities are assigned correct sizes
correct_size = True
printout = np.array(printout)
for i in range(len(printout)):
    true_size = facility_size[i]
    calc_size = np.sum(printout[i])
    if true_size == calc_size:
        print("Facility ", i+1, "\t correct size of", true_size, "\n")
    else:
        print("Facility ", i+1, "\t wrong size.", "True Size = ", true_size, "Calc size = ", calc_size,"\n")
        correct_size = False
if correct_size == True:
    print("all facilities Correct Size")

### Check to ensure same position is not assigned to two facilities
ctr1 = 0
for i in range(len(printout.T)):
    if np.sum(printout.T[i]) >1:
        print("Position ", i+1, " = Overlapped")
    else:
        ctr1+=1

if ctr1 == 81:
    print("No Overlaps")

fig, ax = plt.subplots(figsize=(17, 9))
sns.heatmap(printout,annot = printout, vmax= 1, vmin=0 ).set(title = "Final Matrix")
plt.savefig(f'Images/1HS_matrix_timelimit_{timelimit}_IF_10.png')
np.save(f'1HS_matrix_timelimit_{timelimit}_IF_10.npy',printout)
np.save(f'1HS_layout_timelimit_{timelimit}_IF_10.npy',layout)

t4 = time.time()

print(f'postprocess finished in {t4-t3} seconds')

SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project DEV.